In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import time
import threading
import json
import os
from PIL import Image, ImageTk
import requests
import hashlib
import uuid
import random
import animation_manager

In [ ]:
animate = animation_manager.Animate()


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import time
import threading
import json
import os
from PIL import Image, ImageTk
import requests
import hashlib
import uuid

class DeskAipet:
    def __init__(self, root):
        self.root = root
        self.reminder_thread = threading.Thread(target=self.reminder_loop, daemon=True)
        self.reminder_thread.start()
        self.root.title("桌面宠物")
        self.root.attributes('-topmost', True)
        self.root.overrideredirect(True)
        self.root.attributes('-alpha', 0.9)
        
        # 百度翻译API配置
        self.baidu_appid = '您的百度翻译APPID'
        self.baidu_key = '您的百度翻译密钥'

        # 宠物状态
        self.is_dragging = False
        self.drag_start_x = 0
        self.drag_start_y = 0
        self.current_expression = "close"
        self.expression_frame = 2
        self.animation_active = False
        self.current_animation = None
        self.animation_thread = None

        # 表情图片
        self.expressions = {
            "open1": None,
            "close": None,  
            "open2": None,  
            "denying": None
        }
        
        # 加载表情图片
        self.load_expressions()

        # 文件夹注释存储文件
        self.notes_file = "folder_notes.txt"
        
        # 创建GUI
        self.create_gui()
        
        # 启动宠物动画
        self.start_idle_animation()

    def reminder_loop(self):
        """提醒循环"""
        reminder_interval = 1.5 * 60 * 60  # 1.5小时
        
        while True:
            time.sleep(reminder_interval)
            self.play_talking_animation(3)
            self.show_message("提醒：请保存您的工作文件！")

    def create_gui(self):
        """创建GUI界面"""
        # 主框架
        self.main_frame = tk.Frame(self.root, bg='lightblue', bd=2, relief='solid')
        self.main_frame.pack(padx=5, pady=5)
        
        # 宠物头像
        self.pet_label = tk.Label(self.main_frame, image=self.expressions["close"], bg='lightblue')
        self.pet_label.pack(pady=5)
        
        # 对话框
        self.dialog_frame = tk.Frame(self.main_frame, bg='white', bd=1, relief='solid')
        self.dialog_frame.pack(fill='x', padx=5, pady=5)
        
        self.dialog_text = tk.Text(self.dialog_frame, height=3, width=25, wrap='word', 
                                  bg='white', fg='black', font=('Arial', 10))
        self.dialog_text.pack(padx=5, pady=5)
        self.dialog_text.insert('1.0', "你好！我是月见真。\n我可以帮你翻译英语、提醒保存文件，还能记住文件夹的注释。")
        self.dialog_text.config(state='disabled')
        
        # 按钮框架
        self.button_frame = tk.Frame(self.main_frame, bg='lightblue')
        self.button_frame.pack(fill='x', padx=5, pady=5)
        
        # 动画测试按钮
        self.idle_btn = tk.Button(self.button_frame, text="待机动画", 
                                 command=lambda: self.play_idle_animation(5), bg='lightgreen')
        self.idle_btn.pack(side='left', padx=2, fill='x', expand=True)
        
        self.talk_btn = tk.Button(self.button_frame, text="说话动画", 
                                 command=lambda: self.play_talking_animation(3), bg='lightyellow')
        self.talk_btn.pack(side='left', padx=2, fill='x', expand=True)
        
        self.deny_btn = tk.Button(self.button_frame, text="否认动画", 
                                 command=lambda: self.play_denying_animation(2), bg='lightcoral')
        self.deny_btn.pack(side='left', padx=2, fill='x', expand=True)
        
        # 关闭按钮
        self.close_btn = tk.Button(self.button_frame, text="关闭", 
                                  command=self.root.quit, bg='pink')
        self.close_btn.pack(side='left', padx=2, fill='x', expand=True)
        
        # 绑定拖动事件
        self.pet_label.bind("<ButtonPress-1>", self.start_drag)
        self.pet_label.bind("<B1-Motion>", self.on_drag)
        self.pet_label.bind("<ButtonRelease-1>", self.stop_drag)
        
        # 初始位置
        self.root.geometry("+100+100")

    def start_drag(self, event):
        """开始拖动"""
        self.is_dragging = True
        self.drag_start_x = event.x_root
        self.drag_start_y = event.y_root
    
    def on_drag(self, event):
        """拖动中"""
        if self.is_dragging:
            x = self.root.winfo_x() + (event.x_root - self.drag_start_x)
            y = self.root.winfo_y() + (event.y_root - self.drag_start_y)
            self.root.geometry(f"+{x}+{y}")
            self.drag_start_x = event.x_root
            self.drag_start_y = event.y_root
    
    def stop_drag(self, event):
        """停止拖动"""
        self.is_dragging = False

    def load_expressions(self):
        """加载表情图片"""
        try:
            image_paths = {
                "open1": "face1.1.1.png",     
                "close": "face1.2.1.png", 
                "open2": "face1.3.png",  
                "denying": "face1.4.png"   
            }
            
            for expr_name, file_path in image_paths.items():
                if os.path.exists(file_path):
                    image = Image.open(file_path)
                    image = image.resize((80, 80), Image.Resampling.LANCZOS)
                    self.expressions[expr_name] = ImageTk.PhotoImage(image)
                else:
                    colors = {
                        "open1": "lightblue",
                        "close": "lightblue",
                        "open2": "lightblue",
                        "denying": "lightcoral"
                    }
                    image = Image.new('RGB', (80, 80), color=colors[expr_name])
                    self.expressions[expr_name] = ImageTk.PhotoImage(image)
        except Exception as e:
            print(f"加载表情图片失败: {e}")
            colors = {
                "open1": "lightblue",
                "close": "lightblue",
                "open2": "lightblue",
                "denying": "lightcoral"
            }
            for expr_name, color in colors.items():
                image = Image.new('RGB', (80, 80), color=color)
                self.expressions[expr_name] = ImageTk.PhotoImage(image)

    def set_expression(self, expression):
        """设置宠物表情"""
        if expression != self.current_expression:
            self.current_expression = expression
            self.update_expression_display()
    
    def update_expression_display(self):
        """更新表情显示"""
        if self.current_expression == "talking":
            if self.expression_frame == 1:
                self.pet_label.config(image=self.expressions["open1"])
                self.expression_frame = 2
            else:
                self.pet_label.config(image=self.expressions["open2"])
                self.expression_frame = 1
        elif self.current_expression == "denying":
            self.pet_label.config(image=self.expressions["denying"])
        else:
            self.pet_label.config(image=self.expressions["close"])
            self.expression_frame = 2

    def stop_current_animation(self):
        """停止当前动画"""
        self.animation_active = False
        if self.animation_thread and self.animation_thread.is_alive():
            # 等待动画线程结束
            self.animation_thread.join(timeout=0.5)

    def play_idle_animation(self, duration=5):
        """播放待机动画"""
        def idle_animation():
            self.stop_current_animation()
            self.animation_active = True
            self.current_animation = "idle"
            
            start_time = time.time()
            while self.animation_active and (time.time() - start_time) < duration:
                self.set_expression("close")
                time.sleep(0.5)
                if not self.animation_active:
                    break
                    
            # 动画结束后回到默认状态
            if self.current_animation == "idle":
                self.set_expression("close")
        
        self.animation_thread = threading.Thread(target=idle_animation, daemon=True)
        self.animation_thread.start()

    def play_talking_animation(self, duration=3):
        """播放说话动画"""
        def talking_animation():
            self.stop_current_animation()
            self.animation_active = True
            self.current_animation = "talking"
            
            start_time = time.time()
            while self.animation_active and (time.time() - start_time) < duration:
                self.set_expression("talking")
                time.sleep(0.3)
                if not self.animation_active:
                    break
                    
            # 动画结束后回到默认状态
            if self.current_animation == "talking":
                self.set_expression("close")
        
        self.animation_thread = threading.Thread(target=talking_animation, daemon=True)
        self.animation_thread.start()

    def play_denying_animation(self, duration=2):
        """播放否认动画"""
        def denying_animation():
            self.stop_current_animation()
            self.animation_active = True
            self.current_animation = "denying"
            
            start_time = time.time()
            while self.animation_active and (time.time() - start_time) < duration:
                self.set_expression("denying")
                time.sleep(0.1)  # 否认动画可以更快一些
                if not self.animation_active:
                    break
                    
            # 动画结束后回到默认状态
            if self.current_animation == "denying":
                self.set_expression("close")
        
        self.animation_thread = threading.Thread(target=denying_animation, daemon=True)
        self.animation_thread.start()

    def play_blink_animation(self, duration=1):
        """播放眨眼动画"""
        def blink_animation():
            self.stop_current_animation()
            self.animation_active = True
            self.current_animation = "blink"
            
            # 眨眼动画：快速切换闭眼和睁眼
            blink_count = int(duration * 2)  # 每秒眨眼2次
            for i in range(blink_count):
                if not self.animation_active:
                    break
                self.set_expression("close")
                time.sleep(0.1)
                if not self.animation_active:
                    break
                self.set_expression("open1")
                time.sleep(0.4)
                    
            # 动画结束后回到默认状态
            if self.current_animation == "blink":
                self.set_expression("close")
        
        self.animation_thread = threading.Thread(target=blink_animation, daemon=True)
        self.animation_thread.start()

    def start_idle_animation(self):
        """启动待机动画（默认循环）"""
        def idle_loop():
            while True:
                # 随机待机行为
                import random
                choice = random.randint(1, 10)
                
                if choice <= 7:  # 70%概率普通待机
                    self.play_idle_animation(random.randint(3, 8))
                elif choice <= 9:  # 20%概率眨眼
                    self.play_blink_animation(random.randint(1, 2))
                else:  # 10%概率轻微摇头（说话动画）
                    self.play_talking_animation(random.randint(1, 2))
                
                # 等待当前动画完成
                if self.animation_thread and self.animation_thread.is_alive():
                    self.animation_thread.join()
                time.sleep(1)
        
        threading.Thread(target=idle_loop, daemon=True).start()

    def show_message(self, message):
        """显示消息"""
        self.dialog_text.config(state='normal')
        self.dialog_text.delete('1.0', tk.END)
        self.dialog_text.insert('1.0', message)
        self.dialog_text.config(state='disabled')

def main():
    root = tk.Tk()
    pet = DeskAipet(root)
    root.mainloop()

if __name__ == "__main__":
    main()